In [10]:
from paddlenlp.transformers import BertModel, BertTokenizer,BertForMaskedLM
import paddle
import numpy as np
import ot

In [2]:
model = BertModel.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

[2022-05-18 19:48:21,726] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/bert-base-chinese/bert-base-chinese.pdparams
[2022-05-18 19:48:30,801] [    INFO] - Weights from pretrained model not used in BertModel: ['cls.predictions.decoder_weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.weight', 'cls.predictions.transform.bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.layer_norm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
[2022-05-18 19:48:31,030] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt


In [3]:
test_str1 = '恒大地产2012年拟裁员30%：1月4日，恒大地产（3333.HK）的一位副总裁向记者确认，公司内部发文，2012年将裁员30%。知情人士透露，按步骤初期只裁10%，因为2009年裁员的时候，矛盾冲突比较激烈。该副总称裁员的原因主要是项目销售问题。'
model_inputs1 = tokenizer(test_str1)
model_outputs1 = model(**{k:paddle.to_tensor([v]) for (k,v) in model_inputs1.items()})

In [4]:
pos_list1 = [
    [18,28],
    [103,105],
    [105,109],
    [20,22],
]
for pos in pos_list1:
    print(tokenizer.convert_ids_to_tokens(model_inputs1['input_ids'][pos[0]:pos[1]]))

entity_embs1 = []
for pos in pos_list1:
    temp_maxtrix = paddle.stack([model_outputs1[0][0,idx] for idx in range(pos[0],pos[1])])
    entity_embs1.append(paddle.max(temp_maxtrix,axis=0).numpy())

['恒', '大', '地', '产', '（', '333', '##3', '.', '[UNK]', '）']
['项', '目']
['销', '售', '问', '题']
['地', '产']


In [5]:
test_str2 = '从2021年9月份开始，恒大全国各地项目接连停工率，交楼量不足1万套导致恒大的销量大幅下降；每天都有商票逾期，债券经被停止融资融业务，裁员60%'
model_inputs2 = tokenizer(test_str2)
model_outputs2 = model(**{k:paddle.to_tensor([v]) for (k,v) in model_inputs2.items()})

In [6]:
pos_list2 = [
    [34,43],
    [10,12],
    [18,22],
    [10,12],
]
for pos in pos_list2:
    print(tokenizer.convert_ids_to_tokens(model_inputs2['input_ids'][pos[0]:pos[1]]))

entity_embs2 = []
for pos in pos_list2:
    temp_maxtrix = paddle.stack([model_outputs2[0][0,idx] for idx in range(pos[0],pos[1])])
    entity_embs2.append(paddle.max(temp_maxtrix,axis=0).numpy())
    

['恒', '大', '的', '销', '量', '大', '幅', '下', '降']
['恒', '大']
['接', '连', '停', '工']
['恒', '大']


In [7]:
def _compute_wasserstein_distance(node_embeddings_matrice, sinkhorn=False, 
                                   sinkhorn_lambda=0.01,isImport=False):
    '''
    Generate the Wasserstein distance matrix for the docs embedded 
    in label_sequences
    '''
    # n is the num of docs 
    n = len(node_embeddings_matrice) 
    
    
    M = np.zeros((n,n)) # 

    for i in range(len(node_embeddings_matrice)):
        node_embeddings_matrice[i] = np.nan_to_num(node_embeddings_matrice[i],posinf=1.0,neginf=1e-7,nan=0)
    # Iterate over pairs of docs
    for doc_index_1, doc_1 in enumerate(node_embeddings_matrice):
        for doc_index_2, doc_2 in enumerate(node_embeddings_matrice[doc_index_1:]):
            
            # Get cost matrix
            costs = ot.dist(doc_1, doc_2, metric='sqeuclidean') # (n,m) (6,4)
            # doc_1 has n metric -》 6, bert emb
            # doc_2 has m metric -> 4, bert emb
            
            doc1_dis = np.ones(doc_1.shape[0])/doc_1.shape[0] # uniform dis 1/6 
            doc2_dis = np.ones(doc_2.shape[0])/doc_2.shape[0] # uniform ids 1/4

            if sinkhorn:
                mat = ot.sinkhorn(doc1_dis, doc2_dis, costs, sinkhorn_lambda, numItermax=50) # sinkhorn approx
                M[doc_index_1, doc_index_2 + doc_index_1] = np.sum(np.multiply(mat, costs))
            else:
                M[doc_index_1, doc_index_2 + doc_index_1] = ot.emd2(doc1_dis,doc2_dis, costs)
                    
    M = (M + M.T)
    return M # 是一个n*n的matric，记录了任意两个 doc 之间的相似度

[array([ 1.08359611e+00,  9.34666455e-01,  5.29162943e-01,  1.02064371e+00,
        2.02841565e-01,  1.27457535e+00,  9.91211116e-01,  8.78635168e-01,
        5.56993842e-01,  7.19658315e-01,  6.44357443e-01,  6.93613827e-01,
        5.56697130e-01,  8.16297412e-01,  1.47705972e+00,  9.45996761e-01,
        1.29836965e+00,  4.79693413e-01,  1.17088270e+00,  9.59356844e-01,
        5.88992476e-01,  1.34111810e+00,  5.76335669e-01,  6.37699187e-01,
        1.46280348e+00,  7.13507712e-01,  1.10423946e+00,  5.56758523e-01,
        1.56467819e+00,  4.42626417e-01,  3.96450549e-01,  8.41297746e-01,
        1.12971890e+00,  5.30486703e-01,  1.09905887e+00,  1.16377854e+00,
        5.91352522e-01,  3.49467576e-01,  2.21552849e-01,  1.11294591e+00,
        4.32921767e-01,  2.43149310e-01,  4.20024455e-01,  4.82351482e-01,
        4.28842306e-01,  1.07842958e+00,  5.92611969e-01,  6.59923434e-01,
        1.08535600e+00,  1.14777327e+00,  4.33847934e-01,  1.16961765e+01,
        1.04637170e+00, 

In [ ]:
entity_embs1 = np.array(entity_embs1)
entity_embs2 = np.array(entity_embs2)